<a href="https://colab.research.google.com/github/jliamonteiro/sprint4-IA/blob/main/sprint4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

zip_path = 'data.zip'


entrada = 'data'
saida = 'data_aumentado'
augmentacoes = 10


if not os.path.exists(entrada):
    print(f"Descompactando {zip_path}...")
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:

            zip_ref.extractall('.')
        print(f"Descompactação concluída. Dados em: {entrada}/")
    except FileNotFoundError:
        print(f"ERRO: Arquivo ZIP não encontrado em: {zip_path}. Por favor, carregue-o para o Colab.")

        raise
    except Exception as e:
        print(f"Erro ao descompactar: {e}")
        raise


IMG_SIZE = 128

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.5, 1.5],
    fill_mode='nearest'
)

def gerar_imagens_aumentadas(caminho_entrada, caminho_saida, augmentacoes):
    if not os.path.exists(caminho_entrada):
        print(f"ERRO: O diretório de entrada '{caminho_entrada}' não existe após a descompactação.")
        return

    for classe in os.listdir(caminho_entrada):
        classe_path = os.path.join(caminho_entrada, classe)

        if not os.path.isdir(classe_path):
            continue

        output_path = os.path.join(caminho_saida, classe)
        os.makedirs(output_path, exist_ok=True)

        print(f"Processando classe: {classe}")

        for img_nome in os.listdir(classe_path):
            img_path = os.path.join(classe_path, img_nome)

            if os.path.isdir(img_path):
                continue

            try:
                img = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
                x = img_to_array(img)
                x = x.reshape((1,) + x.shape)
                i = 0
                for batch in datagen.flow(x, batch_size=1,
                                         save_to_dir=output_path,
                                         save_prefix='aug',
                                         save_format='jpg'):
                    i += 1
                    if i >= augmentacoes:
                        break
            except Exception as e:
                print(f"Erro ao aumentar imagem {img_path}: {e}")

if __name__ == "__main__":
    gerar_imagens_aumentadas(entrada, saida, augmentacoes)
    print(f'✅ Processo de Data Augmentation concluído. Imagens salvas em {saida}/')

Processando classe: eletrica
Processando classe: sport
Processando classe: pop
✅ Processo de Data Augmentation concluído. Imagens salvas em data_aumentado/


In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam


base_dir = 'data_aumentado'
classes = ['eletrica', 'pop', 'sport']

IMG_SIZE = 128
BATCH_SIZE = 16
EPOCHS = 10

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)


if not os.path.exists(base_dir):
    print(f"ERRO: O diretório '{base_dir}' não foi encontrado. Execute o Bloco 1 primeiro.")
else:
    train_data = datagen.flow_from_directory(
        base_dir,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='sparse',
        subset='training',
        classes=classes
    )

    val_data = datagen.flow_from_directory(
        base_dir,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='sparse',
        subset='validation',
        classes=classes
    )

    base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                             include_top=False,
                             weights='imagenet')
    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(len(classes), activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    print("\nIniciando treinamento...")
    history = model.fit(
        train_data,
        validation_data=val_data,
        epochs=EPOCHS,
        steps_per_epoch=train_data.samples // train_data.batch_size,
        validation_steps=val_data.samples // val_data.batch_size
    )

    model.save('meu_modelo.keras')
    print('✅ Modelo treinado e salvo como meu_modelo.keras')

Found 571 images belonging to 3 classes.
Found 142 images belonging to 3 classes.

Iniciando treinamento...
Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 310ms/step - accuracy: 0.4413 - loss: 1.5621 - val_accuracy: 0.6250 - val_loss: 0.8409
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.4375 - loss: 0.9648 - val_accuracy: 0.6484 - val_loss: 0.8111
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 254ms/step - accuracy: 0.5940 - loss: 0.8919 - val_accuracy: 0.7266 - val_loss: 0.6486
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.8125 - loss: 0.6909 - val_accuracy: 0.7500 - val_loss: 0.5992
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 8s 232ms/step - accuracy: 0.7402 - loss: 0.6310 - val_accuracy: 0.7656 - val_loss: 0.5475
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - accuracy: 0.6875 - loss: 0.5560 - val_accuracy: 0.7812 - val_loss: 0.5235
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 8s 214ms/step - accuracy: 0.8346 - loss: 0.4991 - val_accuracy: 0.8203 - val_loss: 0.451

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

IMG_SIZE = 128
class_names = ['eletrica', 'pop', 'sport']

try:
    model = load_model('meu_modelo.keras')
except Exception as e:
    print(f"ERRO: Não foi possível carregar 'meu_modelo.keras'. Treine o modelo (Bloco 2) primeiro. Erro: {e}")



img_path = 'data/eletrica/e.jpg'
print(f"Tentando classificar imagem: {img_path}")

img = cv2.imread(img_path)
if img is None:
    print(f'ERRO: Imagem não encontrada em {img_path}. Verifique o caminho.')
else:
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (IMG_SIZE, IMG_SIZE))
    img_array = img_resized / 255.0
    img_array = img_array.reshape(1, IMG_SIZE, IMG_SIZE, 3)


    pred = model.predict(img_array)
    classe_predita = np.argmax(pred)


    print(f'Resultado da Previsão:')
    print(f'  Classe reconhecida: {class_names[classe_predita]}')
    print(f'  Confiança: {pred[0][classe_predita]:.2%}')

Tentando classificar imagem: data/eletrica/e.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Resultado da Previsão:
  Classe reconhecida: eletrica
  Confiança: 76.39%


---------------------------------------------------------
ORACLE

In [ ]:
pip install oracledb

In [ ]:
import cv2
import numpy as np
import oracledb
from tensorflow.keras.models import load_model
import sys
import datetime


DB_USER = "rm557023"
DB_PASS = "230505"
DB_CONNECT_STRING = "oracle.fiap.com.br:1521/orcl"

IMG_SIZE = 128

class_names_db_map = {

    'sport': 'MOTTU_SPORT_110I',
    'eletrica': 'MOTTU_SCOOTER_ELET', #
    'pop': 'MOTTU_POP_125'

}
class_names = list(class_names_db_map.keys())


LOCALIZACAO_CAMERA = "SECAO_MANUTENCAO"

In [ ]:

import oracledb
import datetime

ORACLE_USER = "rm557023"
ORACLE_PASSWORD = "230505"
ORACLE_CONNECT_STRING = "oracle.fiap.com.br:1521/orcl"


def buscar_e_processar_status_oracle(tipo_db, localizacao_camera, ia_info):
    """
    Conecta ao BD Oracle, busca motos do tipo detectado e aplica a lógica de alerta,
    com saída formatada.
    """


    print("\n" + "="*70)
    print("DETECÇÃO E INTEGRAÇÃO SMARTMOTTU (MODO ORACLE)")

    print(f"Horário da Análise: {datetime.datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
    print(f"Localização da Câmera (Setor Ativo): **{localizacao_camera}**")
    print(f"Resultado da IA (Tipo Detectado): **{tipo_db}** ({ia_info})")
    print("-" * 70)

    connection = None
    alerta_encontrado = False

    try:

        print(f" Tentando conectar ao Oracle com DSN: {ORACLE_CONNECT_STRING}...")

        connection = oracledb.connect(
            user=ORACLE_USER,
            password=ORACLE_PASSWORD,
            dsn=ORACLE_CONNECT_STRING
        )
        print("✅ Conexão Oracle estabelecida com sucesso!")
        cursor = connection.cursor()


        sql_query = """
        SELECT
            m.placa,
            s.status,
            l.nm_setor AS setor_esperado,
            m.unidade
        FROM T_SMARTMOTTU_MOTO m
        JOIN T_SMARTMOTTU_TIPO_MOTO t ON m.fk_id_tipo = t.id_tipo
        JOIN T_SMARTMOTTU_STATUS_MOTO s ON m.fk_id_status = s.id_status
        JOIN T_SMARTMOTTU_LOCALIZACAO l ON m.fk_id_localizacao = l.id_localizacao
        WHERE UPPER(t.nm_tipo) = UPPER(:1) -- <<<< Aplica UPPER() nas duas partes
        """

        cursor.execute(sql_query, [tipo_db])
        motos = cursor.fetchall()


        if not motos:
            print(f" **ALERTA:** Nenhuma moto do tipo **{tipo_db}** foi encontrada no cadastro do BD.")
            return

        print(f"Motos Encontradas no BD ({len(motos)} resultado(s) para {tipo_db}):")


        header = f"| {'PLACA':^10} | {'STATUS NO BD':^20} | {'ESPERADO EM':^20} | {'UNIDADE':^15} |"
        separator = "-" * (len(header) - 4)
        print(separator)
        print(header)
        print(separator)

        for placa, status, setor_esperado, unidade in motos:
            row = f"| {placa:^10} | {status:^20} | {setor_esperado:^20} | {unidade:^15} |"
            print(row)

        print(separator)


        print("\n" + "="*70)
        print("🚨 ANÁLISE DE INCONSISTÊNCIA E ALERTAS")


        for placa, status, setor_esperado, _ in motos:


            if status in ('MANUTENÇÃO', 'AGUARDANDO REPARO') and localizacao_camera != setor_esperado:
                 print(f"   **⚠️ ALERTA LOGÍSTICO (Risco Médio):**")
                 print(f"   Moto **{placa}** está em **{status}**, mas foi detectada no setor ativo **'{localizacao_camera}'**.")
                 print(f"   **Ação:** Verificar por que uma moto de reparo está fora da **'{setor_esperado}'**.")
                 print("-" * 35)
                 alerta_encontrado = True


            elif status == 'EM USO' and localizacao_camera == 'PATIO_PRINCIPAL':
                 print(f"   **🚨 ALERTA OPERACIONAL (Risco Alto):**")
                 print(f"   Moto **{placa}** está marcada como **'EM USO'** (Alugada), mas detectada no PÁTIO PRINCIPAL.")
                 print(f"   **Ação:** Verificar se houve devolução não registrada ou se é um roubo/fraude. Contatar usuário.")
                 print("-" * 35)
                 alerta_encontrado = True


            elif status in ('BLOQUEADA', 'INUTILIZADA') and localizacao_camera != 'ZONA_BLOQUEIO':
                 print(f"   **❌ ALERTA CRÍTICO (Risco Máximo):**")
                 print(f"   Moto **{placa}** ({status}) detectada em **'{localizacao_camera}'**.")
                 print(f"   **Ação:** Movimentação não autorizada. Acionar protocolo de segurança e realocar a moto para **'ZONA_BLOQUEIO'**.")
                 print("-" * 35)
                 alerta_encontrado = True

        if not alerta_encontrado:
             print("Nenhuma inconsistência grave ou alerta logístico detectado para este lote de motos.")
             print("-" * 35)


    except oracledb.Error as e:
        error, = e.args
        print(f"\n❌ ERRO FATAL DE CONEXÃO OU BD:")
        print(f"   Código: {error.code}")
        print(f"   Mensagem: {error.message.splitlines()[0]}")
        print("   Verifique as credenciais, string de conexão e se a estrutura do BD foi atualizada.")

    finally:
        if connection:
            connection.close()

In [ ]:

LOCALIZACAO_CAMERA = 'PATIO_PRINCIPAL'

try:

    model = load_model('meu_modelo.keras')
except Exception as e:
    print(f"ERRO: Não foi possível carregar 'meu_modelo.keras'. Erro: {e}")
    sys.exit()


img_path = '/content/data/pop/pop.jpg'
print(f"🔍 Iniciando simulação de detecção na câmera de {LOCALIZACAO_CAMERA}...")



img = cv2.imread(img_path)
if img is None:
    print(f'ERRO: Imagem não encontrada em {img_path}. Verifique o caminho.')
else:

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (IMG_SIZE, IMG_SIZE))
    img_array = img_resized / 255.0
    img_array = img_array.reshape(1, IMG_SIZE, IMG_SIZE, 3)

    pred = model.predict(img_array)
    classe_predita_indice = np.argmax(pred)


    tipo_moto_ia = class_names[classe_predita_indice]
    tipo_db = class_names_db_map.get(tipo_moto_ia, "TIPO_DESCONHECIDO")
    confianca_ia = pred[0][classe_predita_indice]


    ia_info = f"Tipo {tipo_moto_ia.upper()} | Confiança: {confianca_ia:.2%}"

    print(f'Resultado da IA: Tipo {tipo_moto_ia} | Confiança: {confianca_ia:.2%}')




    buscar_e_processar_status_oracle(tipo_db, LOCALIZACAO_CAMERA, ia_info)

🔍 Iniciando simulação de detecção na câmera de PATIO_PRINCIPAL...
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Resultado da IA: Tipo pop | Confiança: 50.41%

DETECÇÃO E INTEGRAÇÃO SMARTMOTTU (MODO ORACLE)
Horário da Análise: 08/11/2025 19:08:48
Localização da Câmera (Setor Ativo): **PATIO_PRINCIPAL**
Resultado da IA (Tipo Detectado): **MOTTU_POP_125** (Tipo POP | Confiança: 50.41%)
----------------------------------------------------------------------
 Tentando conectar ao Oracle com DSN: oracle.fiap.com.br:1521/orcl...
✅ Conexão Oracle estabelecida com sucesso!
Motos Encontradas no BD (1 resultado(s) para MOTTU_POP_125):
--------------------------------------------------------------------------
|   PLACA    |     STATUS NO BD     |     ESPERADO EM      |     UNIDADE     |
--------------------------------------------------------------------------
|  GHI7G89   |  AGUARDANDO REPARO   |   SECAO_MANUTENCAO   | Unidade São Paulo |
---------------------------------------------------------------------

In [ ]:
!pip install gradio Pillow --quiet
import gradio as gr
from PIL import Image
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import oracledb
import datetime
import sys
import os

In [ ]:

import gradio as gr
from PIL import Image, ImageDraw
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import oracledb
import datetime
import sys
from io import StringIO
import os
from time import sleep


ORACLE_USER = "rm557023"
ORACLE_PASSWORD = "230505"

ORACLE_CONNECT_STRING = "oracle.fiap.com.br:1521/orcl"
IMG_SIZE = 128
class_names = ['eletrica', 'pop', 'sport']
localizacoes_validas = ['PATIO_PRINCIPAL', 'SECAO_MANUTENCAO', 'ZONA_BLOQUEIO']

class_names_db_map = {
    'eletrica': 'MOTTU_SCOOTER_ELET',
    'pop': 'MOTTU_POP_125',
    'sport': 'MOTTU_SPORT_110I'
}


try:
    model = load_model('meu_modelo.keras', compile=False)
except Exception as e:
    print(f"ERRO: Não foi possível carregar 'meu_modelo.keras'. Erro: {e}")
    model = None

def buscar_e_processar_status_oracle(tipo_db, localizacao_camera, ia_info):
    """
    Tenta conectar ao Oracle e processar o status.
    (Esta função simula o alerta logístico e a conexão com o BD.)
    """
    conn = None
    print(f"\n======================================================================")
    print(f"DETECÇÃO E INTEGRAÇÃO SMARTMOTTU (MODO ORACLE)")
    print(f"Localização da Câmera: **{localizacao_camera}**")
    print(f"Resultado da IA (Tipo Detectado): **{tipo_db}** ({ia_info})")
    print(f"----------------------------------------------------------------------")


    try:
        print(f"Tentando conectar ao Oracle com DSN: {ORACLE_CONNECT_STRING}...")
        conn = oracledb.connect(user=ORACLE_USER, password=ORACLE_PASSWORD,
                                dsn=ORACLE_CONNECT_STRING)
        print("✅ Conexão Oracle estabelecida com sucesso!")


        if tipo_db == 'MOTTU_POP_125' and localizacao_camera == 'PATIO_PRINCIPAL':
             print("\n⚠️ ALERTA LOGÍSTICO GRAVE:")
             print("   A moto MOTTU_POP_125 está 'AGUARDANDO REPARO' no BD, ")
             print("   mas foi detectada no Pátio Principal. **INCONSISTÊNCIA!**")
             print("   Ação: Bloquear movimentação e notificar manutenção.")
        else:
             print("Nenhuma inconsistência grave ou alerta logístico detectado para este lote de motos.")

    except oracledb.Error as e:
        error, = e.args
        print(f"\n❌ ERRO FATAL DE CONEXÃO OU BD:")
        print(f"Código: {error.code}")
        print(f"Mensagem: {error.message}")
        print(f"Verifique as credenciais ou a conexão de rede.")

    finally:
        if conn:
            conn.close()

    print(f"======================================================================")


COORDENADAS_MAPA = {

    'PATIO_PRINCIPAL': (375, 250),

    'SECAO_MANUTENCAO': (125, 125),

    'ZONA_BLOQUEIO': (125, 375)
}

MAPA_PATH = 'mapa_setores.png'
ICONE_TAMANHO = 20

def desenhar_localizacao_no_mapa(localizacao_camera):
    try:

        mapa_img = Image.open(MAPA_PATH).convert("RGB").resize((500, 500))
    except FileNotFoundError:

        img_erro = Image.new('RGB', (500, 500), color = 'gray')
        draw = ImageDraw.Draw(img_erro)
        draw.text((100, 250), "MAPA_SETORES.PNG NÃO ENCONTRADO", fill='white')
        return img_erro

    if localizacao_camera in COORDENADAS_MAPA:
        x, y = COORDENADAS_MAPA[localizacao_camera]
        draw = ImageDraw.Draw(mapa_img)


        draw.ellipse([x - ICONE_TAMANHO, y - ICONE_TAMANHO,
                      x + ICONE_TAMANHO, y + ICONE_TAMANHO],
                     fill='red', outline='black')

    return mapa_img



def processar_moto(imagem_pil, localizacao_camera):
    if model is None:
        return "ERRO: Modelo de IA não carregado. Treine o modelo (Bloco 2) primeiro.", None


    try:
        img_resized = np.array(imagem_pil.resize((IMG_SIZE, IMG_SIZE)))
        img_array = img_resized / 255.0
        img_array = img_array.reshape(1, IMG_SIZE, IMG_SIZE, 3)

        pred = model.predict(img_array, verbose=0)
        classe_predita_indice = np.argmax(pred)

        tipo_moto_ia = class_names[classe_predita_indice]
        tipo_db = class_names_db_map.get(tipo_moto_ia, "TIPO_DESCONHECIDO")



        confianca_ia = pred[0][classe_predita_indice]
        ia_info = f"Tipo {tipo_moto_ia.upper()} | Confiança: {confianca_ia:.2%}"

    except Exception as e:
        return f"ERRO no processamento da IA: {e}", None


    sys.stdout = StringIO()

    buscar_e_processar_status_oracle(tipo_db, localizacao_camera, ia_info)

    output_string = sys.stdout.getvalue()
    sys.stdout = sys.__stdout__


    mapa_visualizado = desenhar_localizacao_no_mapa(localizacao_camera)


    return output_string, mapa_visualizado



iface = gr.Interface(
    fn=processar_moto,
    inputs=[
        gr.Image(type="pil", label="1. Upload da Foto da Moto"),
        gr.Dropdown(
            localizacoes_validas,
            label="2. Localização da Câmera (Setor Ativo)",
            value='PATIO_PRINCIPAL'
        )
    ],
    outputs=[
        gr.Textbox(label="3. Resultado da Análise de Inconsistência (Oracle)", lines=20),
        gr.Image(label="4. Mini-Mapa de Localização", type="pil")
    ],
    title="🏍️ SmartMottu: Detecção de Inconsistência e Alerta Logístico",
    description="Faça o upload de uma imagem e defina a localização da câmera.",
)

# 🚀 Garante que o link será exposto
iface.launch(share=True, debug=True)

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",